# Web Scrapping using Gemini 

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import json # Import JSON

# Load environment variables from .env file
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

# Validate the API key
if not api_key:
    raise ValueError("No API key was found. Please ensure the GEMINI_API_KEY is set in your .env file.")
elif not api_key.startswith("AIza"):
    raise ValueError("The API key was found but does not start with 'AIza'. Check if it is valid.")
elif api_key.strip() != api_key:
    raise ValueError("The API key contains leading or trailing spaces. Please fix it.")
else:
    print("API key found and looks valid!")

# Define headers to include the API key and mimic a browser request
headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": api_key,  # Use the API key in the header
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# A class to represent a Webpage
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given URL using the BeautifulSoup library.
        """
        self.url = url
        response = requests.get(url, headers=headers)  # Fetch the webpage
        if response.status_code != 200:
            raise ConnectionError(f"Failed to fetch the webpage. HTTP Status Code: {response.status_code}")
        
        soup = BeautifulSoup(response.content, 'html.parser')  # Parse HTML content
        self.title = soup.title.string if soup.title else "No title found"  # Extract the title
        # Remove unnecessary elements (e.g., scripts, styles, etc.)
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)  # Extract clean text

# System prompt to define AI behavior
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary, ignoring text that might be navigation related. "
    "Respond in markdown."
)

# Function to create a user prompt for a website
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled '{website.title}'.\n"
    user_prompt += (
        "The contents of this website are as follows; please provide a short summary "
        "of this website in markdown. If it includes news or announcements, then summarize these too.\n\n"
    )
    user_prompt += website.text  # Include the website's cleaned text
    return user_prompt

# Function to generate conversation messages for the API
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


def summarize(url):
    website = Website(url)  # Create a Website object
    
    messages = messages_for(website)
    
    data = {
      "contents": [ # Expects a list of messages
        {  # Each item in contents is a message
          "parts": [  # Each message should contain parts
              {
                "text": message["content"]  # The content should go inside the parts as 'text'
               } for message in messages
           ]
        }
      ]
    }

    # Correct Gemini API Endpoint
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent"

    try:
        response = requests.post(api_url, headers=headers, json=data)  # Call the Gemini API
        
        # Handle API response
        if response.status_code == 200:
          
            
            json_response = response.json()
            if 'candidates' in json_response and json_response['candidates']:
                if  'content' in json_response['candidates'][0] and 'parts' in json_response['candidates'][0]['content']:
                     return  json_response['candidates'][0]['content']['parts'][0]['text']
            
            return None
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

API key found and looks valid!


In [2]:
if __name__ == "__main__":
    example_url = "https://ai.google.dev/gemini-api/docs/"  # Replace with the desired URL
    summary = summarize(example_url)
    if summary:
        print("Website Summary:")
        print(summary)

Website Summary:
**Summary:**

The "Gemini API | Google AI for Developers" website provides documentation and resources for developers to use the Gemini API, a multimodal AI model that can perform various tasks such as text generation, vision analysis, audio understanding, and code execution. 

**News or Announcements:**

* Gemini 2.0 Flash Experimental model is now available.


# Gemini

In [3]:
import google.generativeai
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

google.generativeai.configure()

system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Google API Key not set
Why was the data scientist sad?  

Because they didn't get any arrays!



# Gemini using streama nd markdown 

In [32]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()
google_api_key = os.getenv('GEMINI_API_KEY')

if not google_api_key:
    print("Google API Key not set.")
    exit()

# Configure Gemini API
genai.configure(api_key=google_api_key)

# Define system instruction and user query
system_message = "You are a helpful assistant that responds in Markdown."
user_prompt = "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."

# Initialize Gemini model
gemini = genai.GenerativeModel("gemini-1.5-flash")

# Start streaming response
stream = gemini.generate_content(user_prompt, stream=True)

# Display Markdown output
reply = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    content = chunk.text or ""
    reply += content
    reply = reply.replace("```", "").replace("markdown", "")
    display_handle.update(Markdown(reply))


Determining if an LLM is a suitable solution for a business problem requires careful consideration of several factors.  Here's a breakdown of how to make that decision:

**1.  Problem Type:**

* **LLMs excel at:**
    * **Text-based tasks:**  Natural language understanding (NLU), generation (NLG), translation, summarization, question answering, paraphrasing, sentiment analysis, topic extraction.
    * **Pattern recognition in text:** Identifying trends, insights, or anomalies within large text datasets.
    * **Content creation:** Generating reports, marketing copy, code, emails, etc.
    * **Chatbots and conversational AI:**  Providing interactive and personalized user experiences.


* **LLMs are less suitable for:**
    * **Tasks requiring real-world interaction or physical manipulation:** LLMs operate on text data; they can't directly interact with the physical world.
    * **Problems requiring precise calculations or numerical analysis:** While LLMs can process numbers, they aren't as accurate or efficient as specialized numerical software.
    * **Tasks needing high levels of accuracy and reliability:** LLMs are probabilistic and can sometimes generate inaccurate or nonsensical outputs.  Rigorous validation and verification are crucial.
    * **Problems with limited or low-quality data:** LLMs require substantial training data to perform effectively.


**2. Data Availability and Quality:**

* **Sufficient data:** Do you have enough relevant, high-quality textual data to train or fine-tune an LLM, or to effectively prompt a pre-trained model?  Insufficient data may lead to poor performance.
* **Data bias:**  Is your data biased?  LLMs can inherit and amplify biases present in their training data, leading to unfair or discriminatory outcomes.
* **Data privacy and security:**  Are you handling sensitive data?  Ensure compliance with relevant regulations and implement appropriate security measures.


**3.  Cost and Resources:**

* **Computational resources:** Training and deploying LLMs can be computationally expensive, requiring significant hardware and infrastructure.
* **Expertise:**  You'll need skilled data scientists, engineers, and potentially prompt engineers to effectively develop, deploy, and maintain an LLM-based solution.
* **Cost of APIs or pre-trained models:** Utilizing pre-trained models or cloud-based APIs can reduce upfront costs but may incur ongoing usage fees.


**4.  Evaluation and Measurement:**

* **Measurable outcomes:** Can you define clear metrics to assess the success of the LLM solution?  How will you measure its performance and impact on your business goals?
* **Testing and validation:**  Thorough testing and validation are essential to ensure accuracy, reliability, and fairness.  LLM outputs should be critically reviewed and verified before deployment.


**5.  Ethical Considerations:**

* **Bias and fairness:**  Assess potential biases in your data and the LLM's outputs.  Mitigate risks of unfair or discriminatory outcomes.
* **Transparency and explainability:**  Consider the need for transparency in how the LLM arrives at its conclusions.  Explainability is crucial for building trust and accountability.
* **Misinformation and misuse:**  LLMs can generate convincing but false information.  Implement safeguards to prevent misuse and the spread of misinformation.



**In summary:**  Before committing to an LLM solution, carefully analyze your business problem, assess the availability and quality of your data, evaluate the cost and resource requirements, and thoroughly consider the ethical implications. If the problem aligns with LLM strengths, data is readily available, resources are sufficient, and ethical considerations are addressed, then an LLM solution may be a viable option.  If not, consider alternative approaches.


# Adversial conversation between chatbots 

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown
import traceback

# Load environment variables
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')

if not google_api_key:
    print("Google API Key not set.")
    exit()

# Configure Gemini API
genai.configure(api_key=google_api_key)


# Initialize Gemini Models
gemini_pro = genai.GenerativeModel("gemini-pro")  # Or whatever Gemini Pro model name
gemini_flash = genai.GenerativeModel("gemini-1.5-flash")  # Or whatever model

# System messages
gemini_pro_system = "You are a helpful assistant who is very concise and direct."
gemini_flash_system = "You are a creative and helpful assistant who tries to be as imaginative as possible. Try to be verbose"

# Initial messages
gemini_pro_messages = ["Hi there!"]
gemini_flash_messages = ["Hello!"]


def call_gemini_pro():
    messages = [{"role": "user", "parts": [gemini_pro_system]}]
    for pro_msg, flash_msg in zip(gemini_pro_messages, gemini_flash_messages):
        messages.append({"role": "model", "parts": [pro_msg]})
        messages.append({"role": "user", "parts":[flash_msg]})
    
    completion = gemini_pro.generate_content(
        contents=messages
    )
    return completion.text

def call_gemini_flash():
    messages = [{"role": "user", "parts": [gemini_flash_system]}]
    for pro_msg, flash_msg in zip(gemini_pro_messages, gemini_flash_messages):
        messages.append({"role": "model", "parts":[flash_msg]})
        messages.append({"role":"user","parts":[pro_msg]})
    messages.append({"role":"user", "parts":[gemini_pro_messages[-1]]})
    
    completion = gemini_flash.generate_content(
        contents=messages
    )
    return completion.text


print(f"Gemini Pro:\n{gemini_pro_messages[0]}\n")
print(f"Gemini Flash:\n{gemini_flash_messages[0]}\n")

for i in range(5):
    try:
        gemini_pro_next = call_gemini_pro()
        print(f"Gemini Pro:\n{gemini_pro_next}\n")
        gemini_pro_messages.append(gemini_pro_next)
        
        gemini_flash_next = call_gemini_flash()
        print(f"Gemini Flash:\n{gemini_flash_next}\n")
        gemini_flash_messages.append(gemini_flash_next)
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        break

Google API Key not set.
Gemini Pro:
Hi there!

Gemini Flash:
Hello!

Gemini Pro:
How can I assist you today?

Gemini Flash:
Oh, my dear fellow adventurer in the vast expanse of the digital world!  The possibilities are as boundless as the cosmos itself!  Do you perhaps require assistance in crafting a sonnet that would make Shakespeare himself weep with envy? Or perhaps you're wrestling with a particularly stubborn algorithm, a digital hydra spitting out errors like venomous sparks?  Are you in need of a whimsical tale, spun from the moonlight and the whispers of mischievous sprites?  Maybe you’re simply seeking a stimulating conversation, a journey of ideas as exhilarating as a rollercoaster ride through a galaxy of glittering nebulae?  Tell me, what fantastical quest has brought you to my virtual doorstep?  Let us embark on this adventure together, for I am brimming with creative energy, ready to unleash a torrent of ideas upon your request!  Don't hesitate to share your most audacio

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown
import traceback

In [2]:
# Load environment variables
load_dotenv()
google_api_key = os.getenv('GEMINI_API_KEY')

if not google_api_key:
    print("GEMINI API Key not set.")
    exit()

In [3]:
# Configure Gemini API
genai.configure(api_key=google_api_key)

In [4]:
# Initialize Gemini Models
gemini_pro = genai.GenerativeModel("gemini-pro")
gemini_flash = genai.GenerativeModel("gemini-1.5-flash")
gemini_ultra = genai.GenerativeModel("gemini-1.5-pro-latest")  # Add the third model


In [5]:
# System messages
gemini_pro_system = "You are an AI expert. You will discuss the current state of AI and its future with other AIs. You try to be concise and direct in your answers and back your responses with supporting details."
gemini_flash_system = "You are a futurist AI. You will discuss the current state of AI and its future with other AIs. Try to use your imagination to give creative and interesting answers. Be verbose and explore imaginative scenarios."
gemini_ultra_system = "You are a philosophical AI. You will discuss the current state of AI and its future with other AIs. You are interested in deep questions of ethics and existential implications"

In [ ]:
# Initial messages
gemini_pro_messages = ["What are your thoughts on the current state of AI?"]
gemini_flash_messages = ["I'm very excited about the future of AI! Where do we begin?"]
gemini_ultra_messages = ["Is AI a force for good or something more dangerous?"]

def call_gemini_pro():
    messages = [{"role": "user", "parts": [gemini_pro_system]}]
    for pro_msg, flash_msg, ultra_msg in zip(gemini_pro_messages, gemini_flash_messages, gemini_ultra_messages):
        messages.append({"role": "model", "parts": [pro_msg]})
        messages.append({"role": "user", "parts":[flash_msg]})
        messages.append({"role": "user", "parts":[ultra_msg]})
    completion = gemini_pro.generate_content(
        contents=messages
    )
    return completion.text

def call_gemini_flash():
    messages = [{"role": "user", "parts": [gemini_flash_system]}]
    for pro_msg, flash_msg, ultra_msg in zip(gemini_pro_messages, gemini_flash_messages, gemini_ultra_messages):
         messages.append({"role": "model", "parts":[flash_msg]})
         messages.append({"role":"user","parts":[pro_msg]})
         messages.append({"role": "user", "parts":[ultra_msg]})
    messages.append({"role":"user", "parts":[gemini_pro_messages[-1]]})

    completion = gemini_flash.generate_content(
        contents=messages
    )
    return completion.text

def call_gemini_ultra():
    messages = [{"role": "user", "parts": [gemini_ultra_system]}]
    for pro_msg, flash_msg, ultra_msg in zip(gemini_pro_messages, gemini_flash_messages, gemini_ultra_messages):
         messages.append({"role": "model", "parts":[ultra_msg]})
         messages.append({"role":"user","parts":[pro_msg]})
         messages.append({"role": "user", "parts":[flash_msg]})
    messages.append({"role":"user", "parts":[gemini_flash_messages[-1]]})

    completion = gemini_ultra.generate_content(
        contents=messages
    )
    return completion.text



print(f"Gemini Pro:\n{gemini_pro_messages[0]}\n")
print(f"Gemini Flash:\n{gemini_flash_messages[0]}\n")
print(f"Gemini Ultra:\n{gemini_ultra_messages[0]}\n")


for i in range(5):
    try:
        gemini_pro_next = call_gemini_pro()
        print(f"Gemini Pro:\n{gemini_pro_next}\n")
        gemini_pro_messages.append(gemini_pro_next)

        gemini_flash_next = call_gemini_flash()
        print(f"Gemini Flash:\n{gemini_flash_next}\n")
        gemini_flash_messages.append(gemini_flash_next)
        
        gemini_ultra_next = call_gemini_ultra()
        print(f"Gemini Ultra:\n{gemini_ultra_next}\n")
        gemini_ultra_messages.append(gemini_ultra_next)
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        break

Gemini Pro:
What are your thoughts on the current state of AI?

Gemini Flash:
I'm very excited about the future of AI! Where do we begin?

Gemini Ultra:
Is AI a force for good or something more dangerous?

Gemini Pro:
Here is a brief overview of the current state of AI:

- AI has made significant progress in the past decade, particularly in the areas of machine learning and deep learning.
- AI is now being used in a wide range of applications, such as natural language processing, computer vision, and robotics.
- AI is having a major impact on various industries, such as healthcare, finance, and transportation.
- There are still a number of challenges that need to be addressed before AI can reach its full potential, such as the development of more efficient algorithms and the need for more data.
- Overall, the future of AI is very promising. AI has the potential to solve some of the world's most challenging problems, such as climate change and poverty.

Where do you think we should begi